Step 1: Setup and Install Necessary Libraries

In [ ]:
!pip install numpy matplotlib opencv-python imgaug

Step 2: Upload Images to Google Colab

In [ ]:
from google.colab import files
import numpy as np
import cv2

# Upload files from your local machine to Google Colab
uploaded = files.upload()

# Load the uploaded images into a list
image_files = list(uploaded.keys())
images = [cv2.imdecode(np.frombuffer(uploaded[image_file], np.uint8), 1) for image_file in image_files]


Step 3: Resize Images to 608x608

In [ ]:
import matplotlib.pyplot as plt

def crop_center(img, target_height, target_width):
    """
    Crop the center part of the image to the target height and width.
    """
    y, x, _ = img.shape
    start_x = x//2 - target_width//2
    start_y = y//2 - target_height//2
    end_x = start_x + target_width
    end_y = start_y + target_height

    # Ensure the cropping dimensions are within the image dimensions
    if start_x < 0 or start_y < 0 or end_x > x or end_y > y:
        print("Image dimensions are smaller than the target crop size.")
        return img  # Return the original image if it's smaller than the crop size

    return img[start_y:end_y, start_x:end_x]

cropped_images = [crop_center(image, 608, 608) for image in images]

# Display the first cropped image to verify
plt.imshow(cv2.cvtColor(cropped_images[0], cv2.COLOR_BGR2RGB))
plt.show()



Step 4: Image Augmentation

In [ ]:
import imgaug as ia
import imgaug.augmenters as iaa

# Define the augmentation sequence
seq = iaa.Sequential([
    iaa.Fliplr(1), # horizontal flips
    iaa.Rot90([1, 2, 3]), # rotate 90, 180, 270 degrees
    iaa.Affine(rotate=(10, 15)) # rotate between 10 and 15 degrees
])

# Apply augmentations
augmented_images = []
for image in cropped_images:
    augmented_images.extend(seq.augment_images([image]*10))

# Display the first augmented image to verify
plt.imshow(cv2.cvtColor(augmented_images[0], cv2.COLOR_BGR2RGB))
plt.show()


Step 5: Download Augmented Images

In [ ]:
from google.colab import files

# Save augmented images and trigger download
for idx, aug_image in enumerate(augmented_images):
    filename = f"aug_{idx}.jpg"
    cv2.imwrite(filename, aug_image)
    files.download(filename)
